In [2]:
# ==============================================================================
# 1. SETUP - INSTALL AND IMPORT LIBS
# ==============================================================================
!pip install langgraph langchain_core==0.1.52 --quiet

from langgraph.graph import StateGraph, START, END
from typing import TypedDict, List

# ==============================================================================
# 2. DEFINE AND BUILD THE SUBGRAPH
# ==============================================================================
# This is a self-contained workflow that we can reuse.
class SubgraphState(TypedDict):
    data: str
    sub_history: List[str]

def sub_step_1(state: SubgraphState) -> dict:
    print("  (Subgraph Step 1 running)")
    return {"sub_history": state["sub_history"] + ["Sub1 Ran"]}

def sub_step_2(state: SubgraphState) -> dict:
    print("  (Subgraph Step 2 running)")
    return {"sub_history": state["sub_history"] + ["Sub2 Ran"]}

def create_subgraph():
    sub_workflow = StateGraph(SubgraphState)
    sub_workflow.add_node("sub1", sub_step_1)
    sub_workflow.add_node("sub2", sub_step_2)
    sub_workflow.add_edge(START, "sub1")
    sub_workflow.add_edge("sub1", "sub2")
    sub_workflow.add_edge("sub2", END)
    return sub_workflow.compile()

# ==============================================================================
# 3. DEFINE AND BUILD THE MAIN (PARENT) GRAPH
# ==============================================================================
class MainGraphState(TypedDict):
    input: str
    main_history: List[str]

def main_step_before(state: MainGraphState) -> dict:
    print("Executing Main Graph Step BEFORE Subgraph")
    return {"main_history": state["main_history"] + ["Before Subgraph"]}

def main_step_after(state: MainGraphState) -> dict:
    print("Executing Main Graph Step AFTER Subgraph")
    return {"main_history": state["main_history"] + ["After Subgraph"]}

# This node is responsible for invoking the subgraph.
def run_subgraph_node(state: MainGraphState) -> dict:
    print("--- Entering Subgraph ---")
    subgraph_app = create_subgraph()
    sub_input_state = {"data": state["input"], "sub_history": []}
    subgraph_result = subgraph_app.invoke(sub_input_state)
    print("--- Exiting Subgraph ---")

    # We can pass the result of the subgraph into the main graph's state.
    subgraph_summary = f"Subgraph completed with history: {subgraph_result['sub_history']}"
    return {"main_history": state["main_history"] + [subgraph_summary]}

# ==============================================================================
# 4. RUN THE MAIN GRAPH
# ==============================================================================
if __name__ == "__main__":
    main_workflow = StateGraph(MainGraphState)
    main_workflow.add_node("before", main_step_before)
    main_workflow.add_node("subgraph_node", run_subgraph_node)
    main_workflow.add_node("after", main_step_after)
    main_workflow.add_edge(START, "before")
    main_workflow.add_edge("before", "subgraph_node")
    main_workflow.add_edge("subgraph_node", "after")
    main_workflow.add_edge("after", END)
    app = main_workflow.compile()

    initial_state = {"input": "start data", "main_history": []}
    final_result = app.invoke(initial_state)

    print("\n--- Final Main Graph State ---")
    for i, step in enumerate(final_result['main_history']):
        print(f"Step {i+1}: {step}")

Executing Main Graph Step BEFORE Subgraph
--- Entering Subgraph ---
  (Subgraph Step 1 running)
  (Subgraph Step 2 running)
--- Exiting Subgraph ---
Executing Main Graph Step AFTER Subgraph

--- Final Main Graph State ---
Step 1: Before Subgraph
Step 2: Subgraph completed with history: ['Sub1 Ran', 'Sub2 Ran']
Step 3: After Subgraph
